In [2]:
import numpy as np
import pandas as pd


In [4]:
train_b = pd.read_csv("../data/train_symp_path.csv")
test_b = pd.read_csv("../data/test_symp_path.csv")
def process(df):
    df['Symptoms'] = df['Symptoms'].str.lower()  # Convert to lower case
    df['Symptoms'] = df['Symptoms'].str.replace('[^\w\s]', '')  # Remove punctuation
    df['Symptoms'] = df['Symptoms'].str.replace('\n', ' ')  # Remove punctuation
    return df
train_label = process(train_b)['Disease']
test_label = process(test_b)['Disease']

In [3]:
text_train = pd.read_csv("../data/X_train_probabilities.csv")
text_test = pd.read_csv("../data/X_test_probabilities.csv")

bert_train = pd.read_csv("../data/train_probabilities.csv")
bert_test = pd.read_csv("../data/test_probabilities.csv")

p_train = pd.read_csv("../data/train_combined_p.csv",header=None)
p_test = pd.read_csv("../data/test_combined_p.csv",header=None)

In [35]:
stacked_probabilities_train = np.hstack((bert_train, text_train))
stacked_probabilities_test = np.hstack((bert_test, text_test))

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded_train = label_encoder.fit_transform(train_label)
y_encoded_test = label_encoder.fit_transform(test_label)

In [37]:
from sklearn.linear_model import LogisticRegression

fusion_model = LogisticRegression().fit(stacked_probabilities_train, y_encoded_train)

In [40]:
coef = fusion_model.coef_[0]
coef

array([ 1.99787493, -0.41376402, -0.97613639, -0.60803293,  1.6478906 ,
       -0.40990194, -0.58609787, -0.65194915])

In [6]:
from sklearn.metrics import classification_report, accuracy_score

y_pred_log = fusion_model.predict(stacked_probabilities_test)

accuracy = accuracy_score(y_encoded_test, y_pred_log)
print(f'Accuracy of the fusion model: {accuracy * 100:.2f}%')


NameError: name 'fusion_model' is not defined

In [42]:
print(classification_report(y_encoded_test, y_pred_log, target_names=label_encoder.classes_))


                                                      precision    recall  f1-score   support

                                   Atopic Dermatitis       0.82      0.97      0.89        32
          Lupus and other Connective Tissue diseases       0.89      0.94      0.91        17
Scabies Lyme Diease and other Infestations and Bites       0.90      0.72      0.80        25
        Seborrheic Keratoses and other Benign Tumors       1.00      0.75      0.86         8

                                            accuracy                           0.87        82
                                           macro avg       0.90      0.84      0.86        82
                                        weighted avg       0.87      0.87      0.86        82



In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2


In [44]:
# Number of classes
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_encoded_train, num_classes=4)
y_val_encoded = to_categorical(y_encoded_test, num_classes=4)

In [48]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)


Epoch 1/10
11/11 [==============================] - 0s 12ms/step - loss: 0.4246 - accuracy: 0.9849 - val_loss: 0.5671 - val_accuracy: 0.8780
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 0.3818 - accuracy: 0.9819 - val_loss: 0.5329 - val_accuracy: 0.8780
Epoch 3/10
11/11 [==============================] - 0s 5ms/step - loss: 0.3337 - accuracy: 0.9879 - val_loss: 0.5053 - val_accuracy: 0.8780
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.3020 - accuracy: 0.9849 - val_loss: 0.4832 - val_accuracy: 0.8780
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 0.2637 - accuracy: 0.9909 - val_loss: 0.4665 - val_accuracy: 0.8780
Epoch 6/10
11/11 [==============================] - 0s 5ms/step - loss: 0.2528 - accuracy: 0.9849 - val_loss: 0.4528 - val_accuracy: 0.8780
Epoch 7/10
11/11 [==============================] - 0s 5ms/step - loss: 0.2212 - accuracy: 0.9909 - val_loss: 0.4420 - val_accuracy: 0.8780
Epoch 8/10
11/11 [=

In [49]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")


3/3 [==============================] - 0s 3ms/step - loss: 0.4282 - accuracy: 0.8780
Validation Accuracy: 0.8780


In [50]:
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)


Epoch 1/10
11/11 [==============================] - 1s 19ms/step - loss: 1.3875 - accuracy: 0.3353 - val_loss: 1.2977 - val_accuracy: 0.3415
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 1.2207 - accuracy: 0.4532 - val_loss: 1.1921 - val_accuracy: 0.3902
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 1.1089 - accuracy: 0.6284 - val_loss: 1.0853 - val_accuracy: 0.8171
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.9634 - accuracy: 0.8308 - val_loss: 0.9678 - val_accuracy: 0.8780
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 0.8135 - accuracy: 0.9335 - val_loss: 0.8387 - val_accuracy: 0.8659
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 0.6723 - accuracy: 0.9819 - val_loss: 0.7133 - val_accuracy: 0.8659
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.5342 - accuracy: 0.9728 - val_loss: 0.5991 - val_accuracy: 0.8659
Epoch 8/10
11/11 [=

In [52]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

3/3 [==============================] - 0s 2ms/step - loss: 0.4348 - accuracy: 0.8537
Validation Accuracy: 0.8537


In [53]:
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)

Epoch 1/10
11/11 [==============================] - 1s 18ms/step - loss: 1.4068 - accuracy: 0.2719 - val_loss: 1.2742 - val_accuracy: 0.5610
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 1.2151 - accuracy: 0.5257 - val_loss: 1.1420 - val_accuracy: 0.8537
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 1.0717 - accuracy: 0.7311 - val_loss: 1.0269 - val_accuracy: 0.8659
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.9376 - accuracy: 0.8640 - val_loss: 0.9061 - val_accuracy: 0.8659
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 0.7805 - accuracy: 0.9245 - val_loss: 0.7840 - val_accuracy: 0.8780
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 0.6688 - accuracy: 0.9245 - val_loss: 0.6758 - val_accuracy: 0.8780
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.5279 - accuracy: 0.9396 - val_loss: 0.5918 - val_accuracy: 0.8659
Epoch 8/10
11/11 [=

In [55]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

3/3 [==============================] - 0s 2ms/step - loss: 0.4822 - accuracy: 0.8659
Validation Accuracy: 0.8659


In [23]:
y_pred_test_nn = fusion_model.predict(stacked_probabilities_test)
predicted_classes = np.argmax(y_pred_test_nn, axis=1)


3/3 [==============================] - 0s 1ms/step


In [24]:
y_encoded_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3])

In [25]:
print(classification_report(y_encoded_test, predicted_classes, target_names=label_encoder.classes_))


                                                      precision    recall  f1-score   support

                                   Atopic Dermatitis       0.79      0.97      0.87        32
          Lupus and other Connective Tissue diseases       0.89      0.94      0.91        17
Scabies Lyme Diease and other Infestations and Bites       0.90      0.72      0.80        25
        Seborrheic Keratoses and other Benign Tumors       1.00      0.62      0.77         8

                                            accuracy                           0.85        82
                                           macro avg       0.90      0.81      0.84        82
                                        weighted avg       0.87      0.85      0.85        82



In [26]:
num_classes = 4  # Assuming y_train is one-hot encoded

fusion_model = Sequential([
    Dense(64, input_shape=(stacked_probabilities_train.shape[1],), activation='relu',kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

fusion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
fusion_model.fit(stacked_probabilities_train, y_train_encoded, validation_data=(stacked_probabilities_test, y_val_encoded), epochs=10, batch_size=32)

Epoch 1/10
11/11 [==============================] - 1s 18ms/step - loss: 1.5984 - accuracy: 0.2356 - val_loss: 1.4611 - val_accuracy: 0.5732
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 1.4009 - accuracy: 0.4441 - val_loss: 1.3201 - val_accuracy: 0.8902
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 1.2398 - accuracy: 0.6526 - val_loss: 1.1985 - val_accuracy: 0.8780
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 1.1150 - accuracy: 0.7915 - val_loss: 1.0884 - val_accuracy: 0.8659
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 0.9567 - accuracy: 0.8792 - val_loss: 0.9739 - val_accuracy: 0.8780
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 0.8355 - accuracy: 0.9547 - val_loss: 0.8533 - val_accuracy: 0.8780
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.6984 - accuracy: 0.9668 - val_loss: 0.7438 - val_accuracy: 0.8780
Epoch 8/10
11/11 [=

In [28]:
val_loss, val_accuracy = fusion_model.evaluate(stacked_probabilities_test, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

3/3 [==============================] - 0s 2ms/step - loss: 0.5441 - accuracy: 0.8780
Validation Accuracy: 0.8780


In [30]:
# average
combined_probabilities = (bert_test + p_test) / 2
y_pred = np.argmax(combined_probabilities, axis=1)
print(f'Accuracy: {accuracy_score(y_encoded_test, y_pred)}')
print(classification_report(y_encoded_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.3902439024390244
                                                      precision    recall  f1-score   support

                                   Atopic Dermatitis       0.39      1.00      0.56        32
          Lupus and other Connective Tissue diseases       0.00      0.00      0.00        17
Scabies Lyme Diease and other Infestations and Bites       0.00      0.00      0.00        25
        Seborrheic Keratoses and other Benign Tumors       0.00      0.00      0.00         8

                                            accuracy                           0.39        82
                                           macro avg       0.10      0.25      0.14        82
                                        weighted avg       0.15      0.39      0.22        82



/Users/sumiko/anaconda3/envs/neuralnetwork/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sumiko/anaconda3/envs/neuralnetwork/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sumiko/anaconda3/envs/neuralnetwork/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [12]:
# vgg
y_pred = np.argmax(bert_test, axis=1)
print(classification_report(y_encoded_test, y_pred, target_names=label_encoder.classes_))

                                                      precision    recall  f1-score   support

                                   Atopic Dermatitis       0.84      0.97      0.90        32
          Lupus and other Connective Tissue diseases       0.84      0.94      0.89        17
Scabies Lyme Diease and other Infestations and Bites       0.90      0.76      0.83        25
        Seborrheic Keratoses and other Benign Tumors       1.00      0.62      0.77         8

                                            accuracy                           0.87        82
                                           macro avg       0.90      0.82      0.85        82
                                        weighted avg       0.87      0.87      0.86        82

